# 📘 Domo Activity Log Extractor

This script automates the retrieval and formatting of user activity logs from a Domo instance.

It supports:

- Authenticating using environment-based credentials
- Backfill logs for a specific month or across multiple months
- Extracting useful metadata (IP address, device, browser)
- Saving logs to CSV files for analysis or auditing


## 📦 Package Installation

This cell contains optional pip install commands for required packages. Uncomment to install them.


In [ ]:
# %pip install domolibrary --upgrade
# %pip install python-dotenv

## ✅ Imports

All necessary libraries are imported here, including those for handling authentication, datetime operations, async requests, and Domo utilities.


In [ ]:
import os
import re
import json
import calendar
import datetime as dt
from dotenv import load_dotenv
from dateutil.relativedelta import relativedelta
from typing import List, Dict, Tuple
import pandas as pd
import httpx

from domolibrary.client import DomoAuth, get_data, DomoError
from domolibrary.client import ResponseGetData as rgd
from domolibrary.utils import convert as dmcv, chunk_execution as dmce


# 🔐 Environment Setup and Authentication

Loads environment variables and sets up Domo token-based authentication.


In [ ]:
# 🔐 Load environment variables and authenticate
assert load_dotenv('.env', override=True)

auth = DomoAuth.DomoTokenAuth(
    domo_access_token=os.environ['DOMO_ACCESS_TOKEN'],
    domo_instance=os.environ['DOMO_INSTANCE']
)

(await auth.who_am_i()).response

## 🧹 Format Log Rows

Helper function that parses raw log entries and extracts structured data such as IP addresses, devices, and browser details.


In [ ]:
# 🧹 Format a single activity log row
# functions are declared inside the function to group them all together.

def format_activity_log_row(row: dict, domo_instance: str) -> dict:
    """Extracts and formats relevant fields from a single log row."""
    def get_ip(additional_comment) -> str:
        return additional_comment.split("IP address ")[-1].split()[0][:-1] if "IP address " in additional_comment else ""
    
    def get_device(additional_comment) -> str: 
        match = re.search(r"from (.*?) device.", additional_comment)
        return match.group(1).split(" ")[-1].strip() if match else ""
    
    def get_browser(additional_comment)-> str: 
        return additional_comment.split("Device app: ")[-1].strip()[:-1] if "Device app: " in additional_comment else ""

    return {
        "Customer": f"{domo_instance}.domo.com",
        "User Id": row.get("userId", ""),
        "Type": row.get("userType", ""),
        "Source Id": str(row.get("userId", "")),
        "Name": row.get("userName", ""),
        "Action": row.get("actionType", ""),
        "Object Type": row.get("objectType", ""),
        "Object Id": row.get("objectId", ""),
        "Object Name": row.get("objectName", ""),
        "IP Address": get_ip(row.get("additionalComment", "")),
        "Event Time": dmcv.convert_epoch_millisecond_to_datetime(row['time']).isoformat(),
        "Device": get_device(row.get("additionalComment", "")),
        "Browser Details": get_browser(row.get("additionalComment", "")),
        "Event Id": "-1",
        "Client Id": str(row.get("clientId", "")) if row.get("clientId") is not None else "",
        "Terminus": "-1"
    }

def format_activity_log_response(response: rgd.ResponseGetData) -> List[Dict]:
    """Formats the entire response from an activity log API call."""
    return [format_activity_log_row(row, response.auth.domo_instance) for row in response.response]

## 📆 Month Range Utility

Returns the start and end of the month for a given offset in months from the current date.


In [ ]:
# 📆 Compute start and end datetime for a month lag
def get_month_range(months_ago: int) -> Tuple[dt.datetime, dt.datetime]:
    """
    Returns start and end datetime for the month 'months_ago' in the past.
    
    Args:
        months_ago (int): Number of months to go back
    
    Returns:
        Tuple[datetime, datetime]: Start and end datetime of the month
    """
    today = dt.datetime.now()
    start = dt.datetime(today.year, today.month, 1) - relativedelta(months=months_ago)
    end_day = calendar.monthrange(start.year, start.month)[1]
    end = dt.datetime(start.year, start.month, end_day, 23, 59, 59)
    return start, end

## 🔍 Fetch Activity Logs

Queries the Domo audit API to retrieve activity log entries for a specified date range, using async HTTP calls.


In [ ]:
# 🔍 Fetch user activity logs from Domo
async def fetch_activity_logs(auth: DomoAuth.DomoAuth, start: int, end: int, limit: int = 1000,
                              max_records: int = None, object_type: str = None, debug: bool = False) -> rgd.ResponseGetData:
    """
    Queries Domo API for user activity logs in a given time range.

    Args:
        auth (DomoAuth): Auth object
        start (int): Start time in epoch milliseconds
        end (int): End time in epoch milliseconds
        limit (int): Max records per request
        max_records (int): Overall record cap
        object_type (str): Specific object type to query
        debug (bool): Enable debug output
    
    Returns:
        ResponseGetData: API response object
    """
    session = httpx.AsyncClient()
    url = f"https://{auth.domo_instance}.domo.com/api/audit/v1/user-audits"
    if object_type and object_type != "ACTIVITY_LOG":
        url += f"/objectTypes/{object_type}"
    
    response = await get_data.looper(
        auth=auth,
        method="GET",
        url=url,
        arr_fn=lambda res: res.response,
        fixed_params={"start": start, "end": end},
        offset_params={"offset": "offset", "limit": "limit"},
        session=session,
        limit=limit,
        maximum=max_records,
        debug_loop=debug,
        debug_api=debug
    )

    if not response.is_success:
        raise DomoError.RouteError(response)

    await session.aclose()
    return response

## 🛠 Extract and Backfill Logs

High-level functions that:

- Retrieve logs for a specific month (based on a lag)
- Backfill multiple months' worth of logs
- Save results to local CSV files


In [ ]:
# 🛠 Extract activity logs for a specific month or backfill period
async def extract_logs_for_month(auth: DomoAuth.DomoAuth, months_ago: int, output_dir: str = './EXPORT') -> List[dict]:
    """
    Fetches and stores logs for the month 'months_ago' ago.
    
    Args:
        auth (DomoAuth): Auth object
        months_ago (int): Lag in months
        output_dir (str): Directory to save results
    
    Returns:
        List[dict]: Formatted activity log entries
    """
    start, end = get_month_range(months_ago)
    partition = f"{end.strftime('%y-%m')} - {auth.domo_instance}"
    start_ms = dmcv.convert_datetime_to_epoch_millisecond(start)
    end_ms = dmcv.convert_datetime_to_epoch_millisecond(end)

    try:
        raw = await fetch_activity_logs(auth, start=start_ms, end=end_ms)
        formatted = format_activity_log_response(raw)
    except DomoError.DomoError as e:
        print(e)
        return []

    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, f"{partition.replace(' ', '')}.csv"), 'w', encoding='utf-8') as f:
        f.write(json.dumps(formatted))

    print(f"✅ Done retrieving logs for {partition}")
    return formatted

async def backfill_logs(auth: DomoAuth.DomoAuth, months: int) -> List[dict]:
    """
    Fetches and saves logs for the past 'months' months.
    
    Args:
        auth (DomoAuth): Auth object
        months (int): Number of months to backfill
    
    Returns:
        List[dict]: Combined activity logs
    """
    results = await dmce.gather_with_concurrency(
        *[extract_logs_for_month(auth, m) for m in range(months)],
        n=1
    )
    return [entry for res in results for entry in res if res]

In [ ]:
# ▶️ Run Backfill
results = await backfill_logs(auth, months=1)
pd.DataFrame(results[:10])